In [1]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('data/scraped_data/reviews_Electronics_5.json.gz')

In [2]:
df.shape

(1689188, 9)

In [3]:
df.head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,1370131200,"06 2, 2013"
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,1290643200,"11 25, 2010"
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,1283990400,"09 9, 2010"
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2.0,"Great grafics, POOR GPS",1290556800,"11 24, 2010"
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1.0,"Major issues, only excuses for support",1317254400,"09 29, 2011"


In [4]:
df['reviewText_len'] = df['reviewText'].apply(lambda x:len(x))

In [7]:
df['reviewText_len'].mean()

634.201737166023

In [5]:
g = df.groupby('overall')

In [10]:
df['asin'].nunique()

63001

In [6]:
for name, group in g:
    print (name,(group['reviewText_len']).count())

1.0 108725
2.0 82139
3.0 142257
4.0 347041
5.0 1009026


In [ ]:
df_meta = getDF('meta_Electronics.json.gz')

In [ ]:
df_meta.head(3)

In [ ]:
df_comb = df.merge?

In [ ]:
df_comb = df.merge(df_meta, how="left", on="asin")

In [ ]:
df_comb.shape

In [ ]:
df_comb.head(3)

In [ ]:
df_sample = df_comb[df_comb['categories'].apply(lambda x:'tv' in (str(x)).lower())]
#df_sample = df_sample[df_sample['brand'].apply(lambda x:'samsung' in (str(x)).lower())]

In [ ]:
df_prod = df_sample[['overall','reviewText','title','brand','categories']]

In [ ]:
df_prod = df_prod.rename(columns={'overall':'rating', 'reviewText':'reviews','title':'product','categories':'category'})
df_prod.count()

In [ ]:
from src.naive_review_analyzer import NaiveReviewAnalyzer
nra = NaiveReviewAnalyzer()
nra.create_bow(df_prod)

for name, group in df_prod.groupby('product'):
    print(name, '\n\t',nra.create_word_list(product=name, rating=[5.0]))

In [ ]:
import ipywidgets
import pyLDAvis.sklearn
from src.LdaReviewAnalyzer import LdaReviewAnalyzer
lda = LdaReviewAnalyzer(df_prod)
lda.fit_transform(num_topics=2, rating=[5])
topics_dict = lda.get_topics()
#lda.display_topic_model()

In [ ]:
print (topics_dict)

In [ ]:
for name, group in df_prod.groupby('product'):
    count = group.count()
    if (count[1] > 100):
        lda = LdaReviewAnalyzer(df_prod)
        lda.fit_transform(num_topics=2, rating=[0,1])
        topics_dict = lda.get_topics()
        lda.display_topic_model()
        print (name)
        print (topics_dict)
        print ("____________")

In [ ]:
from src.w2v_review_analyzer import W2VReviewAnalyzer
import pandas as pd
df = pd.DataFrame()
w2v = W2VReviewAnalyzer(df)
# w2v.fit()

In [ ]:
import pickle
#pickle.dump( w2v.model, open( f'static/w2v.pkl', "wb" ) )
w2v.model = pickle.load(open( f'static/w2v.pkl', "rb" ) )

In [ ]:
w2v.model.wv.vocab


In [ ]:
#w2v.model.wv.most_similar(positive=['samsung',['screen'])

samsung_vec = w2v.model.wv.get_vector('samsung')
broken_vec = w2v.model.wv.get_vector('broken')

w2v.model.wv.most_similar(positive=['ln46c630','cost'])

#w2v.model.wv.similarity('defective','samsung')

In [ ]:
import tensorflow as tf
import os
from tensorflow.contrib.tensorboard.plugins import projector
import warnings
warnings.filterwarnings("ignore")

def visualize(model, output_path):
    meta_file = "w2x_metadata.tsv"
    placeholder = np.zeros((len(model.wv.index2word), 100))

    with open(os.path.join(output_path,meta_file), 'wb') as file_metadata:
        for i, word in enumerate(model.wv.index2word):
            placeholder[i] = model[word]
            # temporary solution for https://github.com/tensorflow/tensorflow/issues/9094
            if word == '':
                print("Emply Line, should replecaed by any thing else, or will cause a bug of tensorboard")
                file_metadata.write("{0}".format('<Empty Line>').encode('utf-8') + b'\n')
            else:
                file_metadata.write("{0}".format(word).encode('utf-8') + b'\n')

    # define the model without training
    sess = tf.InteractiveSession()

    embedding = tf.Variable(placeholder, trainable = False, name = 'w2x_metadata')
    tf.global_variables_initializer().run()

    saver = tf.train.Saver()
    writer = tf.summary.FileWriter(output_path, sess.graph)

    # adding into projector
    config = projector.ProjectorConfig()
    embed = config.embeddings.add()
    embed.tensor_name = 'w2x_metadata'
    embed.metadata_path = meta_file

    # Specify the width and height of a single thumbnail.
    projector.visualize_embeddings(writer, config)
    saver.save(sess, os.path.join(output_path,'w2x_metadata.ckpt'))
    print('Run `tensorboard --logdir={0}` to run visualize result on tensorboard'.format(output_path))

In [ ]:

visualize(w2v.model.wv, 'data/tensor/')

In [ ]:
l = [{'a':4},{'b':2},{'c':3}]

In [ ]:
sorted(l)